# CSCI 2820 - LINEAR ALGEBRA - SPRING 2025

Make sure you fill in any place that says `CODE SOLUTION HERE` or "CODE SOLUTION HERE", as well as your NAMES below:

In [201]:
NAMES = "Sania, Matthew, Alex, Reed"


# FINAL PROJECT (Option 3):  Error-Correcting Codes

In [202]:
## This is a Jupyter notebook for the CU Linear Algebra Final Project.
## Professor Divya E. Vernerey and Robby Green
## Spring 2025

In [203]:
import galois
import pyfinite as pf
import numpy as np
import random

# Final Project: Error-Correcting Codes

Over the course of this project, we will explore the world of finite fields and how they are constructed. Fields are a specific type of algebraic structure, and they form a grounding for linear algebra because they determine the ways we can scale our vectors, and they make up the entries of the vectors themselves. Usually the field we use for linear algebra is the field of real numbers $\mathbb[R]$. However, there are many important applications of linear algebra in which our field of scalars is finite, rather than infinite. One of those applications is in error-correcting codes. 


## Fields


A field is a system of arithmetic with two operations, addition and multiplication, satisfying the following rules: 
1. Addition and multiplication are both commutative: $a+b = b+a$ and $a \times b = b \times a$
2. There is a special element called $0$ such that $0 + a = a + 0 = a$ for all $a$
3. There is a special element called $1$ such that $1 \times a = a \times 1 = a$ for all $a$
4. Addition is invertible (i.e. subtraction is well-defined). For all $a$, there exists $-a$ such that $a + (-a) = 0$
5. Multiplication is invertible (i.e. division is well-defined). For all $a$ except for $a=0$, there exists $a^[-1]$ such that $a \times a^[-1] = 1$
6. Multiplication distributes over addition: $a \times (b+c) = (a \times b) + (a \times c)$

Usually the multiplication sign is dropped and we write "$ab$" instead of $a \times b$


Some examples of fields you might know are the real numbers $\mathbb[R]$ and the complex numbers $\mathbb[C]$. 

Some examples of systems of arithmetic which are not fields are the integers $\mathbb[Z]$ and the natural numbers $\mathbb[N]$. The integers fail to have well-defined division, and the natural numbers fail to have both division and subtraction. 

## The Binary Field

Every vector space requires an underlying field of scalars, which are used as entries of vectors and to scale vectors. Up until now, the field of scalars has typically been $\mathbb[R]$, and occasionally we venture into the world of complex numbers $\mathbb[C]$. Many engineers or applied mathematicians do not go further than these two fields. But there's another field that's very important to computer scientists, and that's the binary field $\mathbb[B] = \[0,1\]$. 

The binary field has only two elements in it: 0 and 1. For the most part, arithmetic works the same way it does in $\mathbb[R]$. Just like in $\mathbb[R]$, we have 
- $0 \times 0 = 0$
- $0 \times 1 = 0$
- $1 \times 0 = 0$
- $1 \times 1 = 1$
- $0 \oplus 0 = 0$
- $0 \oplus 1 = 1$
- $1 \oplus 0 = 1$

However, unlike in $\mathbb[R]$, we have $1 \oplus 1 = 0$ in $\mathbb[B]$. If we interpret $1$ as "true" and $0$ as "false", then addition in $\mathbb[B]$ is the logical XOR operation, and multiplication is the logical AND operation. You should verify for yourself that $\mathbb[B]$ satisfies all of the conditions required to be a field.


### Sanity Check


As long as your set of scalars satisfy the rules of a field, you can still do everything you've learned to do -- you just have to adjust your rules of arithmetic to fit $\mathbb[B]$ instead of using the arithmetic of $\mathbb[R]$. You can do row reduction, diagonalization, and everything else you've learned to do in this class. Feel free to use the boxes below to experiment with this. 

## Other Finite Fields
To do arithmetic in finite fields, we import a special package, `galois`. Galois was the name of the mathematician who discovered and studied finite fields. The binary field $\mathbb[B]$ is also known as $\mathbb[GF](2)$. It turns out that for every prime number $p$, there is a finite field $\mathbb[GF](p)$, wherein the arithmetic is done modulo $p$. We demonstrate below how to instantiate and multiply matrices whose entries are elements of $\mathbb[GF](p)$. Our particular example is in $\mathbb[GF](31)$, and so all matrix arithmetic is done modulo 31.

In [204]:
## this code instantiates a matrix A whose entries are interpreted as elements of GF(31).
GF31 = galois.GF(31)
A = GF31([[17, 25, 18, 8], [7, 9, 21, 15], [6, 16, 6, 30]]);
A

GF([[17, 25, 18,  8],
    [ 7,  9, 21, 15],
    [ 6, 16,  6, 30]], order=31)

In [205]:
# we can also instantiate a matrix B:
B = GF31([[8, 18], [22, 0], [7, 8], [20, 24]]);
B

GF([[ 8, 18],
    [22,  0],
    [ 7,  8],
    [20, 24]], order=31)

In [206]:
# we can multiply matrices using the @ symbol
A@B

GF([[11, 22],
    [19,  3],
    [19,  8]], order=31)

# Error-Correcting Codes

Recall that a code $C$ is a subspace of $\mathbb[F]^n$, where $\mathbb[F]$ is some field of scalars. A binary code is a subspace of $\mathbb[B]^n$. A good code is a code whose codewords are far from each other in terms of edit distance. 

## Hamming Codes

We are going to implement a Hamming code, as well as an encoder and a decoder for it. The encoder is going to be a generator matrix $G$, and the decoder is going to be a parity check matrix $H$. The particular Hamming code we are going to implement is the [15,11] Hamming code -- whose codewords are length 15 and whose datawords are length 11. 

To start, we are going to select a random dataword message $\textbf[m] = m_1 m_2 ... m_[11]$, apply the generator matrix to it, and end up with a random codeword $\textbf[c]$. That is, $G\textbf[m] = \textbf[c]$.  

In [207]:
## Code your solution here. You can use the function np.random.choice or another random function if you prefer. Remember, you need to define and use the GF2() function to make sure your random vector is interpreted as a binary vector and not a real vector. This problem will be easier if you choose G to be a systematic encoding matrix.

# For creating base 2 arrays
GF2 = galois.GF(2)

# 11x11 identity matrix
I_11 = np.eye(11, dtype=int)

# P uses *least significant bit first*
# Rows are 1-16 written in binary besides 1,2,4 and 8 (parity bits)
P = np.array([[1, 1, 0, 0],
                [1, 0, 1, 0],
                [0, 1, 1, 0],
                [1, 1, 1, 0],
                [1, 0, 0, 1],
                [0, 1, 0, 1],
                [1, 1, 0, 1],
                [0, 0, 1, 1],
                [1, 0, 1, 1],
                [0, 1, 1, 1],
                [1, 1, 1, 1]
        ], dtype=int)

# Create G = [I|P]
G = np.concatenate((I_11, P), axis=1)
G = GF2(G)

# Create H = [P^T|I]
I_4 = np.eye(4, dtype=int)
H = np.concatenate((P.T, I_4), axis=1)
H = GF2(H)

# Test with m
m = np.random.choice([0, 1], size=(11, 1))
m = GF2(m)
c = G.T@m
print("c:\n", c)

c:
 [[0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


Now that we have a random codeword $\textbf[c]$, we are going to scramble a random bit of $\textbf[c]$ to get a new string $\textbf[y]$. 

In [208]:
y = c.copy()
i = random.randint(0, 14)

# Flip the bit by adding 1 (XOR)
y[i] += GF2([1])

print("Flipped index:", (y != c).argmax())
print("y:\n", y)

Flipped index: 12
y:
 [[0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]]


Now that we have a scrambled message $\textbf[y]$, we are going to decode it. We need to decode our message using the parity-check matrix. Remember, the result of applying H to a scrambled message will give you the bit-encoding of the index of the string where the error is... so it may be useful to come up with a helper function which translates a bit-string into a number.

In [209]:
## Code your solution here. TODO: Make sure to check that your decoded message is equal to the original random dataword.

# We can find the index of the error by finding the index of the column of H which equals Hy.

# Flatten to a row vector
Hy = H@y.flatten()
for i in range(H.shape[1]):
    if all(H[:,i] == Hy): # Check if column i of H matches Hy
        print(i) # This is the index of the error in y

# TODO: Decode y and fix the error

12


Now, generate a random 15-bit string and try to decode it. Is it possible that this message cannot be decoded? Why or why not? Explain in geometric terms.

In [210]:
## Code your solution here.

(your answer here)

## Reed-Solomon Codes

Instead of using the binary field, we will use a prime field $\mathbb[GF](p)$ here (where arithmetic is done modulo $p$).

Reed-Solomon codes are a coding system in which we interpret the message we wish to send as the coefficients of a polynomial. That is, suppose we have a dataword that we want to send $\textbf[m] = m_0 m_1 m_2 ... m_k$. We will interpret the dataword as the coefficients of the polynomial $$m(x) = m_0 + m_1 x + m_2 x^2 + ... + m_k x^k$$
and then we will evaluate the polynomial on all of the points of a finite field $\mathbb[GF](p) = \mathbb[Z]/p$. The encoded message  that gets sent (i.e. the codeword) is going to be the concatenation of the results of applying $m(x)$ to every single $x \in \mathbb[GF](p)$: $$\textbf[c] = m(0)\hspace[3pt] m(1)\hspace[3pt] m(2)\hspace[3pt] ...\hspace[3pt] m(p-1) $$

In order for this to be a useful code, we need $p > k$. Why is that? 

(your answer here)

### Warm up: naive method
As a warm up, imagine that we receive a message with no errors. That is, we receive a codeword of the form $\textbf[c] = m(0)\hspace[3pt] m(1)\hspace[3pt] m(2)\hspace[3pt] ...\hspace[3pt] m(p-1) $. How do we decode this message to recover the initial dataword $\textbf[m]$?  

(your answer here)

However, since the string that we received might have errors in it, we cannot trust this naive method. But fear not! We can still set up a linear system which will allow us to detect which of the letters in the codeword have been corrupted. Suppose we receive a (possibly corrupted) string $\textbf[y] = y_0y_1 y_2 ... y_[p-1]$. We have a linear system of equations $$y_i =  m(i)$$ for $0 \leq i \leq p-1$. Because there might be an error, some of these equations may not hold. We will introduce an "error-locator polynomial" $e(x)$ with unknown coefficients to fix our system, so that we have $$y_i \cdot e(i) = m(i) \cdot e(i)$$
The error detecting polynomial will be 0 whenever there is an error in the codeword. In other words, whenever an equation from the naive method did not hold, we will multiply both sides by zero to make it hold. <span style="color:red">Since there are at most </span> $d$ <span style="color:red"> errors in </span> $\textbf[y]$, $e(x)$ <span style="color:red"> will have degree at most </span> $d$. This is a nice fix, but unfortunately it makes our system of equations quadratic instead of linear. To remedy this, let $z(x) = m(x) \cdot e(x)$. We can then solve an equivalent system of linear equations $$y_i \cdot e(i) = z(i) $$ where the unknowns in this system are the coefficients of $e(x)$ and $z(x)$. 

Once we have solved for the coefficients of $e(x)$ and $z(x)$, we can use polynomial long division to calculate $$m(x) = \frac[z(x)][e(x)]$$
and from there, the naive method for decoding should carry us across the finish line.

IMPORTANT NOTE: We want to enforce that $e(x)$ is not the zero polynomial, otherwise our system of equations is trivially true always. In order to do this, we'll add $$e_d = 1$$ into our final system of linear equations.

$e_d$ is the coefficient of $x^d$ in $e(x)$: 
$$e(x) = e_0 + e_1 x + e_2x^2 + ... + e_d x^d$$


"<span style="color:red">Since there are at most </span> $d$ <span style="color:red"> errors in </span> $\textbf[y]$, $e(x)$ <span style="color:red"> will have degree at most </span> $d$."

Explain why this statement is true. Utilize the fundamental theorem of algebra and/or the idea of polynomial interpolation.

(your answer here)

By making use of the rank-nullity theorem, what is the distance of a Reed-Solomon code whose codeword length is $n$ and whose dataword length is $k$? 

(your answer here)

To have you implement this Reed-Solomon code, we will use a dataword length of $5$ and a prime field $\mathbb[GF](13) = \mathbb[Z]/13$. Start by generating a random message $m = m_0 m_1 m_2 m_3 m_4$. Encode the message and randomly corrupt 4 of its letters. 

In [211]:
## Code your solution here. Remember, you will need to define and use the function GF13() to ensure that your random vectors are interpreted as prime field vectors and not real vectors.

Now that we have a corrupted message $\textbf[y]$, use its coordinates to set up and solve the linear system $$y_i \cdot e(i) = z(i)$$ and add in the additional constraint $e_d = 1$. Use your favorite linear system solver. What is $d$? 



In [212]:
## Code your solution here.

Once you have solved this linear system, you will have a string of coefficients $z_0 z_1 ... z_[p+d-1]$. Use polynomial long division to calculate $m(x)$. I recommend using `numpy.polydiv()` to do polynomial long division.

In [213]:
## Code your solution here.

Now that you've calculated $m(x)$, find the original dataword $\textbf[m]$

In [214]:
## Code your solution here.

## Bonus
Come up with a systematic encoding scheme for Reed-Solomon codes. Express your encoding scheme as a matrix or as a generator polynomial.

(your answer here)

Explain how to use a Discrete Fourier Transform to decode Reed-Solomon codes. 

(your answer here)

## References

1. Guruswami, Rudra, & Sudan. *Essential Coding Theory*. Online draft, 2023.

2. Documentation for the `galois` package: https://mhostetter.github.io/galois/latest/basic-usage/array-arithmetic/#linear-algebra